### Importing Libraries and Data

In [1]:
import numpy as np 
import pandas as pd
import pandas.io.sql
import pyodbc
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns 
import datetime
from dateutil import parser
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

In [2]:
full_data = pd.read_csv('data.csv').sort_index(axis=1)

In [3]:
full_data.head()

,# Baths,# Bedrooms,# Rooms,Address,Days On Market,Fin SF,List Price,PID,Sales Price,Status,Subdivision
0,2.1,4,8,4802 Meredith Woods RD,27,1825,164950,754-764-6046,165000,CLOSD,Meredith Woods
1,2.1,4,11,4920 Meredith Woods RD,12,1954,244950,755-764-7741,244950,CLOSD,Meredith Woods
2,1.1,3,7,9704 Timber PASS,12,1520,107900,754-765-8344,110000,CLOSD,Bridgewood
3,1.0,3,5,5001 Broad Meadows RD,8,1188,109270,755-765-0931,109270,CLOSD,Bridgewood
4,1.1,3,6,9705 Timber PASS,69,1422,124000,754-765-9220,102000,CLOSD,Bridgewood


### Removing identifer columns

In [4]:
full_data = full_data.drop(['PID', 'Address', 'Status'], axis=1)
full_data.head()

,# Baths,# Bedrooms,# Rooms,Days On Market,Fin SF,List Price,Sales Price,Subdivision
0,2.1,4,8,27,1825,164950,165000,Meredith Woods
1,2.1,4,11,12,1954,244950,244950,Meredith Woods
2,1.1,3,7,12,1520,107900,110000,Bridgewood
3,1.0,3,5,8,1188,109270,109270,Bridgewood
4,1.1,3,6,69,1422,124000,102000,Bridgewood


### Removing Data with Null Values

In [5]:
for col, col_data in full_data.iteritems():
    print "{} have {} data points with null values".format(col, col_data.isnull().sum())

full_data = full_data.dropna()
print "Number of clean data points: {}".format(len(full_data))

# Baths have 0 data points with null values
# Bedrooms have 0 data points with null values
# Rooms have 0 data points with null values
Days On Market have 0 data points with null values
Fin SF have 0 data points with null values
List Price have 0 data points with null values
Sales Price have 0 data points with null values
Subdivision have 0 data points with null values
Number of clean data points: 237


### Removing Data with Negative or Zero Values

In [6]:
zero_or_negative = []
for index, row in full_data.iterrows():
    for col in full_data.columns:
        if row[col] <= 0:
            zero_or_negative.append(index)    

full_data = full_data.drop(full_data.index[zero_or_negative].unique()).reset_index(drop = True)
print "Number of clean data points: {}".format(len(full_data))

Number of clean data points: 236


### Removing Bad # Rooms Data

In [7]:
full_data = full_data[full_data['# Rooms'] - full_data['# Bedrooms'] - full_data['# Baths'] >= 0].reset_index(drop = True)
print "Number of clean data points: {}".format(len(full_data))

Number of clean data points: 234


### Computing the Target

In [8]:
full_data['Target'] = (full_data['List Price'] - full_data['Sales Price'])
full_data.drop(['Sales Price'],axis=1, inplace=True)

In [9]:
full_data.sort_index(axis=1, inplace=True)
full_data.describe()

,# Baths,# Bedrooms,# Rooms,Days On Market,Fin SF,List Price,Target
count,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000
mean,2.023932,3.376068,7.158120,40.213675,1735.910256,209309.217949,3541.119658
std,0.351245,0.543814,1.338036,45.927696,346.664927,44735.681218,8391.740856
min,1.000000,3.000000,5.000000,1.000000,1016.000000,107900.000000,-20500.000000
25%,2.000000,3.000000,6.000000,8.250000,1518.500000,174950.000000,0.000000
50%,2.100000,3.000000,7.000000,23.500000,1662.000000,209950.000000,1500.000000
75%,2.100000,4.000000,8.000000,56.000000,1899.000000,238587.500000,5950.000000
max,3.000000,5.000000,12.000000,384.000000,3135.000000,375950.000000,83000.000000


### Numerical Feature Data Exploration

In [10]:
numerical_features = full_data.drop(['Subdivision', 'Target'], axis=1)

for col, col_data in full_data.iteritems():
    if col in numerical_features:
        print "{} has a correlation of {} with the Target".format(col, round(col_data.corr(full_data['Target']), 3))

# Baths has a correlation of 0.025 with the Target
# Bedrooms has a correlation of 0.123 with the Target
# Rooms has a correlation of 0.168 with the Target
Days On Market has a correlation of 0.223 with the Target
Fin SF has a correlation of 0.186 with the Target
List Price has a correlation of 0.272 with the Target


In [11]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor

feature_to_score = {}

for column in full_data.:
    new_data = full_data.drop(column, axis = 1)
    y_all = full_data[column]
    X_train, X_test, y_train, y_test = train_test_split(new_data, y_all, test_size=0.25, random_state=42)
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(X_train, y_train)
    feature_to_score[column] = regressor.score(X_test, y_test)

for key, value in feature_to_score.iteritems():
    print "The feature {} has a reported prediction score: {}".format(key, value)

ValueError: could not convert string to float: Country Meadows

### Categorical Feature Data Exploration

In [ ]:
selected_features = ['Subdivision']
for col, col_data in full_data.iteritems():
    if col in selected_features:
        print col 
        full_data.groupby(col).mean()['Target'].plot(kind='bar')
        plt.show()
        print full_data.groupby(col).mean()['Target']
        print "\n"

In [ ]:
X_all = full_data.drop(['Target'], axis=1)
X_all.head()

### Preprocess Feature Columns

In [ ]:
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)

    for col, col_data in X.iteritems():
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col) 

        outX = outX.join(col_data)

    return outX

In [ ]:
y_all = full_data['Target']
X_all = preprocess_features(X_all)
X_all.sort_index(axis=1, inplace=True)
X_all.describe()

### Nearest Neighbors Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import grid_search

parameters = [{'weights': ['uniform', 'distance'], 'n_neighbors': range(2, 20)}]
clf  = grid_search.GridSearchCV(KNeighborsRegressor(), param_grid = parameters, cv = 10)
clf.fit(X_all, y_all)
print "Best parameters: {}, Best score: {}".format(clf.best_params_, round(clf.best_score_, 3))